In [100]:
import pandas as pd
import numpy as np
from geopy import Yandex
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time

In [101]:
# поменяем некоторые свойства pandas, чтобы данные полностью выводились на экран
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('mode.chained_assignment', None)

In [102]:
# прочитаем данные и выведем несколько строк получившегося датасета
hotels = pd.read_csv('data/hotels.csv')
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough London W1J 8LT United Kingdom,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF United Kingdom,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Double Room ', ' Stayed 1 night ']",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern Double Room Echo ', ' Stayed 3 nights ', ' Submitted from a mobile device ']",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris France,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotlessly clean excellent location,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standard Room with 1 Double Bed ', ' Stayed 1 night ']",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Breakfasts were just extraordinary Room was clean quiet and very spacious,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Double or Twin Room ', ' Stayed 6 nights ']",516 day,52.385601,4.847060


In [103]:
# выведем информацию о датасете
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [104]:
# проверим получившийся датасет на наличие дубликатов
dublicated_rows = hotels.duplicated().sum()
print(f'Количетво дублирущихся строк: {dublicated_rows}')


Количетво дублирущихся строк: 307


In [105]:
# удалим дубликаты
hotels = hotels.drop_duplicates()

In [106]:
# проверим датасет на пропуски  и выведем признаки, в которых имеются пропуски и количесвто пропусков
hotels.isna().sum()[hotels.isna().sum() > 0]

lat    2448
lng    2448
dtype: int64

In [107]:
# выведем адреса отелей, у которых отсутвуют координаты
mask = (hotels['lat'].isnull()) | (hotels['lng'].isnull())
hotel_addresses = hotels[mask]['hotel_address'].unique()
hotel_addresses


array(['Savoyenstra e 2 16 Ottakring 1160 Vienna Austria',
       '23 Rue Damr mont 18th arr 75018 Paris France',
       'Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria',
       'W hringer Stra e 33 35 09 Alsergrund 1090 Vienna Austria',
       '4 rue de la P pini re 8th arr 75008 Paris France',
       'Sieveringer Stra e 4 19 D bling 1190 Vienna Austria',
       'Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Austria',
       'Bail n 4 6 Eixample 08010 Barcelona Spain',
       'Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria',
       'Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria',
       'Paragonstra e 1 11 Simmering 1110 Vienna Austria',
       'W hringer Stra e 12 09 Alsergrund 1090 Vienna Austria',
       '20 Rue De La Ga t 14th arr 75014 Paris France',
       'Hasenauerstra e 12 19 D bling 1190 Vienna Austria',
       'Sep lveda 180 Eixample 08011 Barcelona Spain',
       'Pau Clar s 122 Eixample 08009 Barcelona Spain',
       'Josefst dter Stra e 22 08 Jos

In [108]:
# воспользуемся библиотекой geocoder для заполения пропусков 

# создадим словарь, где ключом будет адрес, а значениями ключа - долгота и широта

missing_coordinates = {}

for address in hotel_addresses:
    location = Yandex(api_key='xxx').geocode(address)
    missing_coordinates[address] = [location.latitude, location.longitude]

missing_coordinates


{'Savoyenstra e 2 16 Ottakring 1160 Vienna Austria': [48.211783, 16.297308],
 '23 Rue Damr mont 18th arr 75018 Paris France': [48.856663, 2.351556],
 'Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria': [48.210918,
  16.347119],
 'W hringer Stra e 33 35 09 Alsergrund 1090 Vienna Austria': [48.222139,
  16.357513],
 '4 rue de la P pini re 8th arr 75008 Paris France': [48.856663, 2.351556],
 'Sieveringer Stra e 4 19 D bling 1190 Vienna Austria': [48.245935, 16.341397],
 'Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Austria': [48.218213, 16.39411],
 'Bail n 4 6 Eixample 08010 Barcelona Spain': [41.396133, 1.655829],
 'Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria': [48.222139, 16.357513],
 'Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria': [48.206487,
  16.36346],
 'Paragonstra e 1 11 Simmering 1110 Vienna Austria': [48.176599, 16.413999],
 'W hringer Stra e 12 09 Alsergrund 1090 Vienna Austria': [48.222139,
  16.357513],
 '20 Rue De La Ga t 14th arr 75014 Paris Fran

In [109]:
# заполним отсутствующие координаты
for address in hotel_addresses:
    mask1 = (hotels['lat'].isnull()) | (hotels['lng'].isnull())
    mask2 = hotels['hotel_address'] == address
    hotels.loc[mask1 & mask2, 'lat'] = missing_coordinates[address][0]
    hotels.loc[mask1 & mask2, 'lng'] = missing_coordinates[address][1]

In [110]:
# создадим словарь уникальных тэгов - ключом будет сам тэг, а значение - сколько раз этот тэг встречается
unique_tags = {}

for tags in hotels['tags'].values:
    tags_text = tags.strip('[]')
    tags_list = tags_text.split(', ')
    tags_list = [element.strip("'").strip() for element in tags_list]
    for tag in tags_list:
        if tag in unique_tags.keys():
            unique_tags[tag] += 1
        else:
            unique_tags[tag] = 1



In [111]:
# сделаем из получившегося словаря датасет
tags_df = pd.DataFrame(unique_tags.items(), columns=['tag_name', 'number_of_entries'])
display(tags_df.sort_values(by='number_of_entries', ascending=False, ignore_index=True).head(15))

# выведем самый популярный тэг
mask = tags_df['number_of_entries'] == tags_df['number_of_entries'].max()
most_popular_tag = tags_df[mask].at[0, 'tag_name']
print(f'Наиболее популяный тег: {most_popular_tag}')

# Из тегов выясните, на сколько ночей чаще всего останавливаются путешественники в отелях.
mask = tags_df['tag_name'].str.contains('Stayed')
most_popular_number_of_nights = tags_df[mask].sort_values(by='number_of_entries', ascending=False, ignore_index=True).at[0, 'tag_name']
print(f'Путешественники чаще всего останавливались на одну ночь, так как самый популяный тег: {most_popular_number_of_nights}')

,tag_name,number_of_entries
0,Leisure trip,313353
1,Submitted from a mobile device,230608
2,Couple,189046
3,Stayed 1 night,145296
4,Stayed 2 nights,100176
5,Solo traveler,81166
6,Stayed 3 nights,71940
7,Business trip,61934
8,Group,49057
9,Family with young children,45810


Наиболее популяный тег: Leisure trip
Путешественники чаще всего останавливались на одну ночь, так как самый популяный тег: Stayed 1 night


In [112]:
# Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
# Приведем данные в датасете к формату даты
hotels['review_date'] = hotels['review_date'].astype('datetime64[ns]')

first_review_date = hotels['review_date'].dt.date.min()
last_review_date = hotels['review_date'].dt.date.max()

print(f'Дата первого отзыва: {first_review_date}\nДата последнего отзыва: {last_review_date}')


Дата первого отзыва: 2015-08-04
Дата последнего отзыва: 2017-08-03


In [113]:
# Сколько уникальных названий отелей представлено в наборе данных?
hotels_inique  = hotels['hotel_name'].nunique()

print(f'Количество уникальных названий отелей: {hotels_inique}')

Количество уникальных названий отелей: 1492


In [114]:
display(hotels.info())
display(hotels.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 386496 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386496 non-null  object        
 1   additional_number_of_scoring                386496 non-null  int64         
 2   review_date                                 386496 non-null  datetime64[ns]
 3   average_score                               386496 non-null  float64       
 4   hotel_name                                  386496 non-null  object        
 5   reviewer_nationality                        386496 non-null  object        
 6   negative_review                             386496 non-null  object        
 7   review_total_negative_word_counts           386496 non-null  int64         
 8   total_number_of_reviews                     386496 non-null  int64         
 9 

None

hotel_address                                 0
additional_number_of_scoring                  0
review_date                                   0
average_score                                 0
hotel_name                                    0
reviewer_nationality                          0
negative_review                               0
review_total_negative_word_counts             0
total_number_of_reviews                       0
positive_review                               0
review_total_positive_word_counts             0
total_number_of_reviews_reviewer_has_given    0
reviewer_score                                0
tags                                          0
days_since_review                             0
lat                                           0
lng                                           0
dtype: int64

In [115]:
# создадим дополнительные признаки hotel_city и hotel_country

hotels['hotel_country'] = hotels['hotel_address'].apply(lambda x: x.split()[-1] if x.split()[-2] != 'United' else ' '.join(x.split()[-2:]))
hotels['hotel_city'] = hotels['hotel_address'].apply(lambda x: x.split()[-2] if 'London' not in x else 'London')

In [116]:
# создадим признак, в которому укажем, является ли постоялец резидентом  в стране нахождения отеля
hotels['is_resident'] = hotels['hotel_country'].str.strip() == hotels['reviewer_nationality'].str.strip()

In [117]:
# Приведем данные в датасете к формату даты
hotels['review_date'] = hotels['review_date'].astype('datetime64[ns]')

# создадим новые признаки из даты, а именно признак дня недели (review_weekday), месяца (review_month) и года (review_year)

hotels['review_weekday'] = hotels['review_date'].dt.day_name()
hotels['review_month'] = hotels['review_date'].dt.month_name()
hotels['review_year'] = hotels['review_date'].dt.year

# дополнительно создадим признак сезона
seasons_dict = {
    'December': 'winter', 
    'January': 'winter', 
    'February': 'winter',
    'March': 'spring',
    'April' : 'spring',
    'May': 'spring',
    'June': 'summer',
    'July': 'summer',
    'August': 'summer',
    'September': 'autumn',
    'October': 'autumn',
    'November': 'autumn'
}

hotels['review_season'] = hotels['review_month'].apply(lambda x: seasons_dict[x])


In [118]:
# создадим признак, где бы отображалось количество проведенных в отеле ночей. Для этого используем информацию из тегов, имеющих вид Stayed XX nigths

# создадим словарь уникальных тэгов - ключом будет сам тэг, а значение - сколько раз этот тэг встречается
unique_tags = {}

for tags in hotels['tags'].values:
    tags_text = tags.strip('[]')
    tags_list = tags_text.split(', ')
    tags_list = [element.strip("'").strip() for element in tags_list]
    for tag in tags_list:
        if tag in unique_tags.keys():
            unique_tags[tag] += 1
        else:
            unique_tags[tag] = 1

# сделаем из получившегося словаря датасет
tags_df = pd.DataFrame(unique_tags.items(), columns=['tag_name', 'number_of_entries'])

# выведем все теги, где упоминается, сколько ночей постоялец провел в отеле.
mask1 = tags_df['tag_name'].str.contains('Stayed')
tags_df[mask1].sort_values(by='number_of_entries', ascending=False, ignore_index=True)['tag_name'].unique()

array(['Stayed 1 night', 'Stayed 2 nights', 'Stayed 3 nights',
       'Stayed 4 nights', 'Stayed 5 nights', 'Stayed 6 nights',
       'Stayed 7 nights', 'Stayed 8 nights', 'Stayed 9 nights',
       'Stayed 10 nights', 'Stayed 11 nights', 'Stayed 12 nights',
       'Stayed 14 nights', 'Stayed 13 nights', 'Stayed 15 nights',
       'Stayed 16 nights', 'Stayed 17 nights', 'Stayed 18 nights',
       'Stayed 19 nights', 'Stayed 21 nights', 'Stayed 20 nights',
       'Stayed 30 nights', 'Stayed 27 nights', 'Stayed 22 nights',
       'Stayed 28 nights', 'Stayed 23 nights', 'Stayed 26 nights',
       'Stayed 24 nights', 'Stayed 25 nights', 'Stayed 29 nights'],
      dtype=object)

In [119]:
# при помощи регулярных выражений вычленим количество проведенных в отеле ночей в отдельный признак

reqex = 'Stayed\s\d{1,2}'
hotels['stayed_nights'] = pd.to_numeric(hotels['tags'].str.findall(reqex).str.get(0).str.findall('\d{1,2}').str.get(0), errors='coerce')
nan_values = hotels['stayed_nights'].isnull().sum()
print(f'Количество постояльцев, которые не указали количество проведенных в отеле ночей: {nan_values}')

Количество постояльцев, которые не указали количество проведенных в отеле ночей: 146


In [120]:
# заполним пропуски в данных о количестве проведенных ночей, используя среднее значение
hotels['stayed_nights'] = hotels['stayed_nights'].fillna(hotels['stayed_nights'].mean())

# поменяем тип данных на целочисленный
hotels['stayed_nights'] = hotels['stayed_nights'].astype('int')



In [121]:
# добавим анализ отзывов
nltk.downloader.download('vader_lexicon')

sent_analyzer = SentimentIntensityAnalyzer()

hotels['positive_review_score'] = hotels['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['compound'])
hotels['negative_review_score'] = hotels['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x)['compound'])




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [122]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 386496 entries, 0 to 386802
Data columns (total 27 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386496 non-null  object        
 1   additional_number_of_scoring                386496 non-null  int64         
 2   review_date                                 386496 non-null  datetime64[ns]
 3   average_score                               386496 non-null  float64       
 4   hotel_name                                  386496 non-null  object        
 5   reviewer_nationality                        386496 non-null  object        
 6   negative_review                             386496 non-null  object        
 7   review_total_negative_word_counts           386496 non-null  int64         
 8   total_number_of_reviews                     386496 non-null  int64         
 9 

In [123]:
# закодируем столбцы 'hotel_country', 'hotel_city', 'is_resident', 'review_weekday', 'review_month', 'review_year', 'review_season' при помощи pd.get_dummies

columns_to_code = ['hotel_country', 'hotel_city', 'is_resident', 'review_weekday', 'review_month', 'review_year', 'review_season']

hotels = pd.get_dummies(hotels, columns=columns_to_code, dtype=int)


In [124]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 386496 entries, 0 to 386802
Data columns (total 60 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386496 non-null  object        
 1   additional_number_of_scoring                386496 non-null  int64         
 2   review_date                                 386496 non-null  datetime64[ns]
 3   average_score                               386496 non-null  float64       
 4   hotel_name                                  386496 non-null  object        
 5   reviewer_nationality                        386496 non-null  object        
 6   negative_review                             386496 non-null  object        
 7   review_total_negative_word_counts           386496 non-null  int64         
 8   total_number_of_reviews                     386496 non-null  int64         
 9 

In [125]:
# удалим текстовые признаки

columns_to_dprop = [col for col in hotels.columns if hotels[col].dtypes == 'object']
hotels = hotels.drop(columns_to_dprop, axis=1)

# также удалим признак review_date 

hotels = hotels.drop('review_date', axis=1)

# выведем информацию об оставшихся признаках

hotels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 386496 entries, 0 to 386802
Data columns (total 52 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                386496 non-null  int64  
 1   average_score                               386496 non-null  float64
 2   review_total_negative_word_counts           386496 non-null  int64  
 3   total_number_of_reviews                     386496 non-null  int64  
 4   review_total_positive_word_counts           386496 non-null  int64  
 5   total_number_of_reviews_reviewer_has_given  386496 non-null  int64  
 6   reviewer_score                              386496 non-null  float64
 7   lat                                         386496 non-null  float64
 8   lng                                         386496 non-null  float64
 9   stayed_nights                               386496 non-null  int32  
 10  p

In [126]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [127]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [128]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [129]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [130]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.12604077183404644
